In [44]:
from sympy import *
k_iso, theta, delta =  symbols("k_iso, theta, delta")
Ex, Ey =  symbols("Ex, Ey")

In [45]:
Q      = Matrix([[cos(theta), sin(theta)],[-sin(theta), cos(theta)]]) 
E_cart = Matrix([[Ex],[Ey]]) 

In [46]:
# Transform E to material coordinates
E_rot  = Q*E_cart
E_rot

Matrix([
[ Ex*cos(theta) + Ey*sin(theta)],
[-Ex*sin(theta) + Ey*cos(theta)]])

In [47]:
# Flux in material coordinates (transverse)
K     = Matrix([[k_iso, 0],[0, k_iso/delta]])
q_rot = -K *E_rot
q_rot

Matrix([
[       -k_iso*(Ex*cos(theta) + Ey*sin(theta))],
[-k_iso*(-Ex*sin(theta) + Ey*cos(theta))/delta]])

In [48]:
# Transform back to Cartesian
q_cart = Q.transpose() * q_rot
q_cart

Matrix([
[-k_iso*(Ex*cos(theta) + Ey*sin(theta))*cos(theta) + k_iso*(-Ex*sin(theta) + Ey*cos(theta))*sin(theta)/delta],
[-k_iso*(Ex*cos(theta) + Ey*sin(theta))*sin(theta) - k_iso*(-Ex*sin(theta) + Ey*cos(theta))*cos(theta)/delta]])

In [49]:
# Cartesian conductivity tensor
kxx = -q_cart[0].diff(Ex)
kxy = -q_cart[0].diff(Ey)
kyx = -q_cart[1].diff(Ex)
kyy = -q_cart[1].diff(Ey)
print(julia_code(kxx))
print(julia_code(kxy))
print(julia_code(kyx))
print(julia_code(kyy))

k_iso .* cos(theta) .^ 2 + k_iso .* sin(theta) .^ 2 ./ delta
k_iso .* sin(theta) .* cos(theta) - k_iso .* sin(theta) .* cos(theta) ./ delta
k_iso .* sin(theta) .* cos(theta) - k_iso .* sin(theta) .* cos(theta) ./ delta
k_iso .* sin(theta) .^ 2 + k_iso .* cos(theta) .^ 2 ./ delta


In [50]:
def stencil(kxxW, kxyW, kxxE, kxyE,   kyyS, kyxS, kyyN, kyxN,     kxySW, kxySE, kxyNW, kxyNE, kxxSW, kxxSE, kxxNW, kxxNE, kyySW, kyySE, kyyNW, kyyNE):
    uSW, uS, uSE, uW, uC, uE, uNW, uN, uNE = symbols('uSW, uS, uSE, uW, uC, uE, uNW, uN, uNE')

    dx, dy = symbols('dx,dy')

    dy = dx

    ExSW = (uS-uSW)/dx 
    ExW  = (uC-uW)/dx 
    ExNW = (uN-uNW)/dx 
    ExSE = (uSE-uS)/dx 
    ExE  = (uE-uC)/dx 
    ExNE = (uNE-uN)/dx 

    EySW = (uW-uSW)/dy 
    EyS  = (uC-uS)/dy 
    EyNW = (uNW-uW)/dy 
    EySE = (uE-uSE)/dy 
    EyN  = (uN-uC)/dy 
    EyNE = (uNE-uE)/dy

    ExS = Rational(1,4)*(ExW + ExE + ExSW + ExSE)
    ExN = Rational(1,4)*(ExW + ExE + ExNW + ExNE)
    EyW = Rational(1,4)*(EyS + EyN + EySW + EyNW)
    EyE = Rational(1,4)*(EyS + EyN + EySE + EyNE)

    ExS = Rational(1,2)*(ExSW + ExSE)
    ExN = Rational(1,2)*(ExNW + ExNE)
    EyW = Rational(1,2)*(EySW + EyNW)
    EyE = Rational(1,2)*(EySE + EyNE)

    # qxW = -kxxW*ExW -kxyW*EyW
    # qxE = -kxxE*ExE -kxyE*EyE
    # qyS = -kyyS*EyS -kyxS*ExS
    # qyN = -kyyN*EyN -kyxN*ExN

    # kxyS = kyxS
    # kxyN = kyxN
    # qxW = -kxxW*ExW - 0*Rational(1,4)*(kxyS*EyS  + kxyN*EyN + kxySW*EySW + kxyNW*EyN)
    # qxE = -kxxE*ExE - 0*Rational(1,4)*(kxyS*EyS  + kxyN*EyN + kxySE*EySE + kxyNE*EyN)
    # qyS = -kyyS*EyS - 0*Rational(1,4)*(kxyW*ExW  + kxyE*ExE + kxySW*ExSW + kxySE*ExS)
    # qyN = -kyyN*EyN - 0*Rational(1,4)*(kxyW*ExW  + kxyE*ExE + kxyNW*ExNW + kxyNE*ExN)

    DxNE = 1/2*(  (uNE+uE) - (uN+uC)) / dx
    DyNE = 1/2*(  (uNE+uN) - (uE+uC)) / dx
    DxNW = 1/2*( -(uNW+uW) + (uN+uC)) / dx
    DyNW = 1/2*(  (uNW+uN) - (uW+uC)) / dx
    DxSE = 1/2*(  (uSE+uE) - (uS+uC)) / dx
    DySE = 1/2*( -(uSE+uS) + (uE+uC)) / dx
    DxSW = 1/2*( -(uSW+uW) + (uS+uC)) / dx
    DySW = 1/2*( -(uSW+uS) + (uW+uC)) / dx

    qxW = -1/2*( kxxSW*DxSW + kxxNW*DxNW + kxySW*DySW + kxyNW*DyNW )
    qxE = -1/2*( kxxSE*DxSE + kxxNE*DxNE + kxySE*DySE + kxyNE*DyNE )
    qyS = -1/2*( kxySW*DxSW + kxySE*DxSE + kyySW*DySW + kyySE*DySE )
    qyN = -1/2*( kxyNW*DxNW + kxyNE*DxNE + kyyNW*DyNW + kyyNE*DyNE )

    return -(qxE - qxW)/dx -(qyN - qyS)/dy 

In [51]:
kxxW, kxyW, kxxE, kxyE = symbols('kxxW, kxyW, kxxE, kxyE')
kyyS, kyyN, kyxS, kyxN = symbols('kyyS, kyyN, kyxS, kyxN')
kxx1, kxy1, kyx1, kyy1 = symbols('kxx1, kxy1 kyx1, kyy1')
kxx2, kxy2, kyx2, kyy2 = symbols('kxx2, kxy2, kyx2, kyy2')

kxySW, kxySE, kxyNW, kxyNE = symbols('kxySW, kxySE, kxyNW, kxyNE')
kxxSW, kxxSE, kxxNW, kxxNE = symbols('kxxSW, kxxSE, kxxNW, kxxNE')
kyySW, kyySE, kyyNW, kyyNE = symbols('kyySW, kyySE, kyyNW, kyyNE')
uSW, uS, uSE, uW, uC, uE, uNW, uN, uNE = symbols('uSW, uS, uSE, uW, uC, uE, uNW, uN, uNE')


In [52]:
# N/S
nodeN = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy1, kxy1, kxy2, kxy2, kxx1, kxx1, kxx2, kxx2, kyy1, kyy1, kyy2, kyy2 )
nodeS = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy2, kxy2, kxy1, kxy1, kxx2, kxx2, kxx1, kxx1, kyy2, kyy2, kyy1, kyy1 )
a=nodeS.diff(uN)
b=nodeN.diff(uS)
(a-b).simplify()

0

In [53]:
# W/E
nodeE = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy1, kxy2, kxy1, kxy2, kxx1, kxx2, kxx1, kxx2, kyy1, kyy2, kyy1, kyy2 )
nodeW = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy2, kxy1, kxy2, kxy1, kxx2, kxx1, kxx2, kxx1, kyy2, kyy1, kyy2, kyy1 )
a=nodeE.diff(uW)
b=nodeW.diff(uE)
(a-b).simplify()

0

In [54]:
# NE/SW
nodeNE = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy1, kxy2, kxy2, kxy2, kxx1, kxx2, kxx2, kxx2, kyy1, kyy2, kyy2, kyy2 )
nodeSW = stencil(kxxW, kxyW, kxxE, kxyE,  kyyS, kyxS, kyyN, kyxN,       kxy2, kxy1, kxy1, kxy1, kxx2, kxx1, kxx1, kxx1, kyy2, kyy1, kyy1, kyy1 )
a=nodeNE.diff(uSW)
b=nodeSW.diff(uNE)
(a-b).simplify()

0